# Live AQI Monitoring

This notebook demonstrates how to monitor real-time air quality data using cpcbfetch.

In [ ]:
from cpcbfetch import LiveAQIClient
import json
import time
from datetime import datetime
import pandas as pd

## 1. Get Your Location

First, let's find your approximate location using IP-based geolocation.

In [ ]:
# Initialize the Live AQI client
client = LiveAQIClient()

# Get your location
location = client.get_system_location()
print("Your Location:")
print(json.dumps(location, indent=2))

## 2. Find Nearest Monitoring Station

Find the closest air quality monitoring station to your location.

In [ ]:
# Find nearest station
nearest_station = client.get_nearest_station()
print("Nearest Monitoring Station:")
print(json.dumps(nearest_station, indent=2))

# Extract station ID for future use
station_id = nearest_station.get('station_id')
station_name = nearest_station.get('station_name', 'Unknown')
print(f"\nMonitoring Station ID: {station_id}")
print(f"Station Name: {station_name}")

## 3. Get Current Air Quality Data

Fetch the latest air quality measurements from the nearest station.

In [ ]:
# Get current AQI data
current_data = client.get_live_aqi_data(station_id=station_id)

print(f"Current Air Quality at {station_name}:")
print(json.dumps(current_data, indent=2))

# Extract key metrics
if current_data:
    aqi = current_data.get('AQI', 'N/A')
    category = current_data.get('Category', 'Unknown')
    pm25 = current_data.get('PM2.5', 'N/A')
    pm10 = current_data.get('PM10', 'N/A')
    
    print(f"\n📊 Summary:")
    print(f"   AQI: {aqi}")
    print(f"   Category: {category}")
    print(f"   PM2.5: {pm25} μg/m³")
    print(f"   PM10: {pm10} μg/m³")

## 4. Continuous Monitoring

Set up a simple monitoring loop to track air quality changes over time.

**Note:** This will run continuously. Stop the cell when you want to end monitoring.

In [ ]:
# Monitoring configuration
CHECK_INTERVAL = 300  # Check every 5 minutes (300 seconds)
MAX_ITERATIONS = 12   # Run for 1 hour (12 * 5 minutes)

# Store monitoring data
monitoring_data = []

print(f"Starting air quality monitoring...")
print(f"Station: {station_name}")
print(f"Checking every {CHECK_INTERVAL // 60} minutes")
print(f"Will run for {(MAX_ITERATIONS * CHECK_INTERVAL) // 60} minutes\n")
print("-" * 80)

try:
    for i in range(MAX_ITERATIONS):
        # Get current data
        data = client.get_live_aqi_data(station_id=station_id)
        
        if data:
            # Add timestamp
            data['timestamp'] = datetime.now().isoformat()
            data['station_name'] = station_name
            monitoring_data.append(data)
            
            # Display current reading
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            aqi = data.get('AQI', 'N/A')
            category = data.get('Category', 'Unknown')
            
            print(f"[{timestamp}] AQI: {aqi} | Category: {category}")
        else:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Failed to fetch data")
        
        # Wait before next check (except on last iteration)
        if i < MAX_ITERATIONS - 1:
            time.sleep(CHECK_INTERVAL)

except KeyboardInterrupt:
    print("\n\n⚠️  Monitoring stopped by user")

print("\n" + "-" * 80)
print(f"Monitoring complete. Collected {len(monitoring_data)} readings.")

## 5. Analyze Monitoring Data

Convert the collected data to a DataFrame for analysis.

In [ ]:
if monitoring_data:
    # Create DataFrame
    df = pd.DataFrame(monitoring_data)
    
    print("Monitoring Data Summary:")
    print(f"\nShape: {df.shape}")
    print(f"\nColumns: {df.columns.tolist()}")
    
    # Display first few rows
    print("\nFirst few readings:")
    display(df.head())
    
    # Basic statistics
    if 'AQI' in df.columns:
        print("\nAQI Statistics:")
        print(df['AQI'].describe())
else:
    print("No monitoring data collected.")

## 6. Visualize Monitoring Data

Create a time series plot of the AQI readings.

In [ ]:
import matplotlib.pyplot as plt

if monitoring_data and 'AQI' in df.columns:
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Plot AQI over time
    plt.figure(figsize=(12, 6))
    plt.plot(df['timestamp'], df['AQI'], marker='o', linewidth=2, markersize=8)
    plt.xlabel('Time')
    plt.ylabel('AQI')
    plt.title(f'Air Quality Monitoring - {station_name}')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Check for significant changes
    if len(df) > 1:
        aqi_change = df['AQI'].iloc[-1] - df['AQI'].iloc[0]
        print(f"\n📈 AQI Change: {aqi_change:+.1f}")
        
        if abs(aqi_change) > 50:
            print("⚠️  Significant air quality change detected!")
else:
    print("Insufficient data for visualization.")

## 7. Set Up Alerts

Create a simple alert system for unhealthy air quality levels.

In [ ]:
def check_air_quality_alert(aqi_value):
    """
    Check AQI and return alert level.
    
    AQI Categories:
    - 0-50: Good
    - 51-100: Satisfactory
    - 101-200: Moderate
    - 201-300: Poor
    - 301-400: Very Poor
    - 401+: Severe
    """
    if aqi_value <= 50:
        return "✅ Good", "Air quality is good. Enjoy outdoor activities!"
    elif aqi_value <= 100:
        return "🟢 Satisfactory", "Air quality is acceptable."
    elif aqi_value <= 200:
        return "🟡 Moderate", "Sensitive individuals should limit prolonged outdoor exposure."
    elif aqi_value <= 300:
        return "🟠 Poor", "Everyone may begin to experience health effects. Reduce outdoor activities."
    elif aqi_value <= 400:
        return "🔴 Very Poor", "Health alert! Everyone should avoid outdoor activities."
    else:
        return "🆘 Severe", "Health emergency! Stay indoors and avoid any outdoor exposure."

# Check current AQI
if current_data and 'AQI' in current_data:
    current_aqi = current_data['AQI']
    alert_level, message = check_air_quality_alert(current_aqi)
    
    print(f"\nCurrent AQI: {current_aqi}")
    print(f"Alert Level: {alert_level}")
    print(f"Recommendation: {message}")

## 8. Save Monitoring Data

Export the monitoring data for future reference.

In [ ]:
if monitoring_data:
    # Save to CSV
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"aqi_monitoring_{station_name.replace(' ', '_')}_{timestamp_str}.csv"
    
    df.to_csv(filename, index=False)
    print(f"Monitoring data saved to: {filename}")
    
    # Also save as JSON
    json_filename = filename.replace('.csv', '.json')
    df.to_json(json_filename, orient='records', indent=2)
    print(f"JSON data saved to: {json_filename}")
else:
    print("No data to save.")

## 9. Get AQI for Specific Coordinates

You can also get AQI data for any specific location by providing coordinates.

In [ ]:
# Example: Get AQI for Mumbai coordinates
mumbai_lat = 19.0760
mumbai_lon = 72.8777

coords = (mumbai_lat, mumbai_lon)

# Find nearest station to these coordinates
mumbai_station = client.get_nearest_station(coords=coords)
print("Nearest station to Mumbai:")
print(json.dumps(mumbai_station, indent=2))

# Get AQI data for that location
mumbai_aqi = client.get_live_aqi_data(coords=coords)
print("\nCurrent AQI in Mumbai:")
print(json.dumps(mumbai_aqi, indent=2))

## Conclusion

This notebook demonstrated:
- Getting your current location
- Finding nearest monitoring stations
- Fetching real-time air quality data
- Setting up continuous monitoring
- Creating alerts based on AQI levels
- Visualizing and saving monitoring data

You can extend this by:
- Adding email/SMS alerts
- Monitoring multiple stations
- Creating dashboards with real-time updates
- Integrating with IoT devices